In [ ]:
#importing dataset from kaggle
import kagglehub
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Download already complete (69155672 bytes).
Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mlg-ulb/creditcardfraud/versions/3


In [ ]:
import pandas as pd

df = pd.read_csv(path + "/creditcard.csv")
df.shape

(284807, 31)

In [ ]:
df.shape
positive = df[df.Class == 1]
negative = df[df.Class == 0]
print(f"positive percentage: {len(positive)/len(df)*100}")
print(f"negative percentage: {len(negative)/len(df)*100}")

positive percentage: 0.1727485630620034
negative percentage: 99.82725143693798


In [ ]:
#SMOTE for balancing data
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(df.drop('Class', axis=1), df['Class'])
balanced_data = pd.concat([X, y], axis=1)
balanced_data.to_csv('balanced_data.csv', index=False)
positive_examples = balanced_data[balanced_data.Class == 1]
negative_examples = balanced_data[balanced_data.Class == 0]
print(f"positive percentage: {len(positive_examples)/len(balanced_data)*100}")
print(f"negative percentage: {len(negative_examples)/len(balanced_data)*100}")

positive percentage: 50.0
negative percentage: 50.0


In [ ]:
balanced_data.shape
missing = df.isnull().sum()
print(missing)

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [ ]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
#Apply LSTM to the data
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np



In [ ]:
X = balanced_data.drop('Class', axis=1)
y = balanced_data['Class']
X_np = X.to_numpy()
X_expanded = X_np[:, np.newaxis, :]
y_np = y.to_numpy()

def create_dataset(X, y, look_back=10):
  dataX, dataY = [], []
  for i in range(len(X)-look_back-1):
    a = X[i:(i+look_back), :]
    dataX.append(a)
    dataY.append(y[i + look_back])
  return np.array(dataX), np.array(dataY)

look_back = 10  # Number of previous time steps to consider
X, y = create_dataset(X_np, y_np, look_back)



# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Long for classification
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=False)

In [ ]:
X_np.shape

(568630, 30)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

# Instantiate the model
input_size = X_train.shape[2]  # Number of features
hidden_size = 128  # Number of hidden units in LSTM
num_layers = 2  # Number of LSTM layers
output_size = 2  # Binary classification (fraud/no fraud)

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

In [ ]:
for data, targets in train_loader:
    print(data.shape)
    print(targets.shape)
    break

torch.Size([64, 10, 30])
torch.Size([64])


In [ ]:
#mounting the drive for saving the model later
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 80  # Adjust as needed
for epoch in range(num_epochs):
    for data, targets in train_loader:
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/80], Loss: 0.0042
Epoch [2/80], Loss: 0.0342
Epoch [3/80], Loss: 0.4174
Epoch [4/80], Loss: 0.0010
Epoch [5/80], Loss: 0.0019
Epoch [6/80], Loss: 0.0033
Epoch [7/80], Loss: 0.0937
Epoch [8/80], Loss: 0.0025
Epoch [9/80], Loss: 0.0716
Epoch [10/80], Loss: 0.0006
Epoch [11/80], Loss: 0.0023
Epoch [12/80], Loss: 0.0031
Epoch [13/80], Loss: 0.0017
Epoch [14/80], Loss: 0.0006
Epoch [15/80], Loss: 0.0008
Epoch [16/80], Loss: 0.1627
Epoch [17/80], Loss: 0.0037
Epoch [18/80], Loss: 0.0019
Epoch [19/80], Loss: 0.0015
Epoch [20/80], Loss: 0.0007
Epoch [21/80], Loss: 0.0016
Epoch [22/80], Loss: 0.0005
Epoch [23/80], Loss: 0.0119
Epoch [24/80], Loss: 0.0034
Epoch [25/80], Loss: 0.0040
Epoch [26/80], Loss: 0.0024
Epoch [27/80], Loss: 0.0021
Epoch [28/80], Loss: 0.0026
Epoch [29/80], Loss: 0.0311
Epoch [30/80], Loss: 0.0012
Epoch [31/80], Loss: 0.0012
Epoch [32/80], Loss: 0.0005
Epoch [33/80], Loss: 0.0034
Epoch [34/80], Loss: 0.0008
Epoch [35/80], Loss: 0.0009
Epoch [36/80], Loss: 0.0012
E

In [ ]:
#saving the model to drive
save_dir = '/content/drive/My Drive/saved_models'  # Change 'saved_models' to your preferred folder name
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
model_save_path = os.path.join(save_dir, 'lstm_model_weights.pth')
torch.save(model.state_dict(), model_save_path)


In [ ]:
#testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, targets in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: 99.88%


In [ ]:
#loading the model from drive
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load(model_save_path))
model.eval()